In [1]:
import numpy as np
import fasttext
import pandas as pd
import random
import itertools
from resources.tokTT import CommentTokenizer as CT
from resources.basicIO import InputOutput as IO
from resources.basicIO import InputOutput as IO
from resources.filterLang import FilterLanguage as FL
from resources.fasttext_transformer import classify
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.neural_network import MLPClassifier
import fasttext
import pandas as pd
from scipy import spatial
from scipy.spatial import distance
from sklearn import svm
from sklearn import metrics
from sklearn.linear_model import SGDClassifier
#from sklearn.linear_model import LassoLars
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import MultinomialNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from sklearn.neighbors import KNeighborsClassifier
import copy
import scipy
from sklearn.metrics import precision_recall_fscore_support
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import validation_curve
import gc
from sklearn.metrics import DetCurveDisplay, RocCurveDisplay
from resources.expansion import Expand_U
import math

[nltk_data] Downloading package wordnet to C:\Users\AJAY
[nltk_data]     BISWAS\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
df_sample = pd.read_csv('datasets/random_sample_data.csv')

In [3]:
# remove unnecessary comments
df_sample = df_sample.drop([x for x in range(len(df_sample)) if int(
    df_sample['label'][x]) != 0 and int(df_sample['label'][x]) != 1])

df_sample.tail()

,raw_comment,tokenized_comment,label
4477,all the flags are sikh religious flags. they d...,all the flag be sikh religious flag they do no...,1
4478,Salute the farmers,salute the farmer,0
4479,Let's also remember how many farmers lost thei...,let be also remember how many farmer lose thei...,0
4480,ABP news you guys dont have heart 8 farmers go...,abp news you guy dont have heart 8 farmer get ...,0
4481,These new laws actually give the power to the ...,these new law actually give the power to the f...,1


In [4]:
comments = df_sample['tokenized_comment']
labels = df_sample['label']

### Weak Supervision

In [5]:
def n_gram_count(sentence: str, n_gram_phrase: str):
    return sentence.count(n_gram_phrase)

### Modified TF-IDF

In [6]:

def IDF(corpus, unique_words):
   idf_dict = {}
   N = len(corpus)
   for i in unique_words:
     count = 0
     for sen in corpus:
       if n_gram_count(sen,i) != 0:
         count = count+1
       idf_dict[i] = (math.log((1+N)/(count+1)))+1
   return idf_dict


def TF(sentence, n_gram_phrase, unique_words):
    """count of t in d / number of words in d

       each phrase in unique_words is considered as a word
    """
    freq = n_gram_count(sentence, n_gram_phrase)
    total_words = len(sentence.split())
    remove = 0
    for word in unique_words:
      freq_word = n_gram_count(sentence, word)
      word_count = freq_word * len(word.split())
      freq_word = word_count - freq_word
      remove += freq_word
    total_words -= remove
    return freq/total_words

def corpus_unique(corpus, n_gram_words):
  """ returns the total unique words in corpus """
  all_sentences = (' '.join(corpus))
  all_words = list(set(all_sentences.split()))
  all_words.extend(n_gram_words)
  
  return list(set(all_words))


In [7]:
n_gram_words = ['wheat and rice','good for farmer','stand with','700 farmer',
               'so call','so called','salute to','free market','rich farmer',
                'we support','adani','ambani','khalistan','flag','victory',
                'loss','corporates','middleman','middle man','khalistani']
corpus_words = corpus_unique(list(comments), n_gram_words)

### Make Feature Vector

In [8]:
F = np.zeros((len(df_sample), len(n_gram_words)))

comments_list = list(comments)

labels_str = list(labels)
labels_int = [int(i) for i in labels_str]

for i in range(len(comments_list)):
    for j in range(len(n_gram_words)):
        tf_idf_ij = TF(comments_list[i], n_gram_words[j], corpus_words)
        F[i][j] = tf_idf_ij

In [9]:
X_raw = df_sample['tokenized_comment']
y = df_sample['label']


In [10]:
model_2 = fasttext.load_model("models/ft_unsupervised_N_2.bin")
model_3 = fasttext.load_model("models/ft_unsupervised_N_3.bin")


In [11]:
X_2 = [model_2[x] for x in X_raw]
X_3 = [model_3[x] for x in X_raw]
mdl = model_2
X = np.hstack((X_2,F))
print(X.shape)

(3254, 320)


In [12]:
print(X[0])

[ 0.02240662  0.02617648 -0.06105003  0.02469208  0.03085181 -0.01056053
 -0.00797712 -0.00668243 -0.00024438  0.05027063 -0.01142706  0.0500012
 -0.00720274 -0.01717712 -0.0148936   0.03807749  0.06606516 -0.02020561
 -0.02553094 -0.03268506  0.00292853  0.01527047 -0.02881388  0.0093856
  0.04100512 -0.04387803  0.01977714 -0.00414409 -0.01651457  0.01412005
 -0.02032061 -0.01771477  0.01930476  0.06880365 -0.02744946 -0.04024558
  0.04712401 -0.03462878 -0.05357217 -0.02888196 -0.00759002 -0.03640257
  0.01321779 -0.03102638  0.03284127 -0.08591734 -0.00766853 -0.00345979
  0.03274981  0.01110621 -0.00768263 -0.0961987  -0.02196658  0.00567446
  0.02912362  0.04259184 -0.0095796   0.04649972  0.06637276 -0.03027004
  0.04684062 -0.02667232  0.03759938 -0.00304972  0.0258259  -0.07651334
 -0.03152761  0.00332509  0.02768348  0.02377841  0.03742892  0.07488141
  0.01502203 -0.004222   -0.05046839 -0.02997868 -0.030106    0.06006132
 -0.0021668   0.02372488 -0.00618488 -0.02659548  0.0

In [13]:

X_train, X_test, y_train, y_test = train_test_split(X,
                                                    df_sample['label'], test_size=0.2,
                                                    random_state=37,
                                                    stratify=df_sample['label'])


In [14]:
N_SAMPLES = 1000

classifiers = {
    "SVM": make_pipeline(StandardScaler(), svm.SVC(probability=True, kernel='rbf')),
    "KNN": KNeighborsClassifier(n_neighbors=19),
    "LR": make_pipeline(StandardScaler(), LogisticRegression(solver='lbfgs', penalty='l2', C=0.05, max_iter=10000, random_state=2)),
    "MLP": make_pipeline(StandardScaler(), MLPClassifier(max_iter=1000, activation='logistic'))
}


### Classification

In [15]:
X_seed, X_expand, y_seed, y_expand = train_test_split(X_train,
                                                      y_train, test_size=0.99,
                                                      random_state=40,
                                                      stratify=y_train)


In [16]:
#print('model ', 'algorithm, ', 'batch_size, ', 'best_amount, ', 'expanded_size, ', 'outp_algorithm, ', 'c_accuracy')

algorithms = ['LR', 'SVM', 'KNN', 'MLP']
batch_sizes = [20]
best_amounts = [5]
big_results = []
big_confusion = []
    
X = np.hstack((X_2, X))

for algorithm in algorithms:
    alg = classifiers[algorithm]

    for batch_size in batch_sizes:

        for best_amount in best_amounts:

            X_uncertain_exp_u, y_uncertain_exp_u, X_certain_exp_u, y_certain_exp_u = Expand_U(mdl,
                                                                                                alg,
                                                                                                list(
                                                                                                    X_seed),
                                                                                                list(
                                                                                                    y_seed),
                                                                                                list(
                                                                                                    X_expand),
                                                                                                list(
                                                                                                    y_expand),
                                                                                                batch_size,
                                                                                                best_amount
                                                                                                )

            # classify
            c_model = alg.fit(X_uncertain_exp_u, y_uncertain_exp_u)
            y_predict = c_model.predict(X_test)
            c_accuracy = metrics.accuracy_score(y_test, y_predict)
            c_prfsw = list(precision_recall_fscore_support(
                y_test, y_predict, average='weighted'))
            c_prfsmi = list(precision_recall_fscore_support(
                y_test, y_predict, average='micro'))

            l1 = [mdl, algorithm, batch_size, best_amount, len(X_uncertain_exp_u), c_accuracy]
            l1.extend(c_prfsw)
            l1.extend(c_prfsmi)

            big_results.append(l1)
            big_confusion.append(metrics.confusion_matrix(
                y_test, c_model.predict(X_test)))
            #print(md[1], ',', algorithm, ',', batch_size, ',', best_amount, ',', len(X_uncertain_exp_u), ',', outp_algorithm, ',', c_accuracy)


In [17]:
print(big_results)

[[<fasttext.FastText._FastText object at 0x00000241203E6B20>, 'LR', 20, 5, 666, 0.7112135176651305, 0.7098625098523159, 0.7112135176651305, 0.7102056808543334, None, 0.7112135176651305, 0.7112135176651305, 0.7112135176651304, None], [<fasttext.FastText._FastText object at 0x00000241203E6B20>, 'SVM', 20, 5, 666, 0.7265745007680492, 0.7282775147126072, 0.7265745007680492, 0.727179100169996, None, 0.7265745007680492, 0.7265745007680492, 0.7265745007680491, None], [<fasttext.FastText._FastText object at 0x00000241203E6B20>, 'KNN', 20, 5, 666, 0.6728110599078341, 0.6791301001728643, 0.6728110599078341, 0.6742484154313274, None, 0.6728110599078341, 0.6728110599078341, 0.6728110599078341, None], [<fasttext.FastText._FastText object at 0x00000241203E6B20>, 'MLP', 20, 5, 666, 0.7434715821812596, 0.7446413293346944, 0.7434715821812596, 0.7439140834230619, None, 0.7434715821812596, 0.7434715821812596, 0.7434715821812596, None]]
